# Enable Databricks Mosaic AI Gateway features

This notebook shows how to enable and use Databricks Mosaic AI Gateway features to manage and govern models from providers, such as OpenAI and Anthropic. 

In this notebook, you use the Model Serving and AI Gateway API to accomplish the following tasks:

- Create and configure an endpoint for OpenAI GPT-4o-Mini.
- Enable AI Gateway features including usage tracking, inference tables, guardrails, and rate limits. 
- Set up invalid keywords and personally identifiable information (PII) detection for model requests and responses.
- Implement rate limits for model serving endpoints.
- Configure multiple models for A/B testing.
- Enable fallbacks for failed requests.

If you prefer a low-code experience, you can create an external models endpoint and configure AI Gateway features using the Serving UI ([AWS](https://docs.databricks.com/ai-gateway/configure-ai-gateway-endpoints.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/ai-gateway/configure-ai-gateway-endpoints) | [GCP](https://docs.databricks.com/gcp/ai-gateway/configure-ai-gateway-endpoints)).

In [0]:
%pip install -U -qqqq openai langchain databricks-langchain
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text(name="catalog", defaultValue="bo_cheng_dnb_demos", label="catalog")
dbutils.widgets.text(name="schema", defaultValue="clio", label="schema")
dbutils.widgets.text(name="secret_scope", defaultValue="dbdemos", label="secret_scope")
dbutils.widgets.text(
    name="openai_api_key_value", defaultValue="", label="openai_api_key_value"
)

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

DATABRICKS_HOST = w.config.host

secret_scope_name = dbutils.widgets.get("secret_scope")

# if needed create a secret scope
if secret_scope_name != "dbdemos":
    w.secrets.create_scope(scope=secret_scope_name)
else:
    print(f"Using existing secret scope: {secret_scope_name}")

In [0]:
# name of model serving endpoint
ENDPOINT_NAME = "gpt-4o"

# catalog and schema for inference tables
CATALOG_NAME = dbutils.widgets.get("catalog")
SCHEMA_NAME = dbutils.widgets.get("schema")

# openai API key in Databricks Secrets
SECRETS_SCOPE = dbutils.widgets.get("secret_scope")
SECRETS_KEY = "openai_api_key"

# if you need to add an OpenAI API key, you can do so with:
if dbutils.widgets.get("openai_api_key_value") == "":
    print(f"no openai_api_key_value provided, using existing secret")
else:
    w.secrets.put_secret(
        scope=SECRETS_SCOPE,
        key=SECRETS_KEY,
        string_value=dbutils.widgets.get("openai_api_key_value"),
    )

## Create a model serving endpoint for OpenAI GPT-4o-Mini

The following creates a model serving endpoint for GPT-4o Mini *without* AI Gateway enabled. First, you define a helper function for creating and updating the endpoint:

In [0]:
import requests
import json
import time
from typing import Optional


def configure_endpoint(
    name: str,
    databricks_token: str,
    config: dict,
    host: str,
    endpoint_path: Optional[str] = None,
):
    base_url = f"{host}/api/2.0/serving-endpoints"

    if endpoint_path:
        # Update operation
        api_url = f"{base_url}/{name}/{endpoint_path}"
        method = requests.put
        operation = "Updating"
    else:
        # Create operation
        api_url = base_url
        method = requests.post
        operation = "Creating"

    headers = {
        "Authorization": f"Bearer {databricks_token}",
        "Content-Type": "application/json",
    }

    print(f"{operation} endpoint...")
    response = method(api_url, headers=headers, json=config)

    if response.status_code == 200:
        return response.json()
    else:
        print(
            f"Failed to {operation.lower()} endpoint. Status code: {response.status_code}"
        )
        return response.text

Next, write a simple configuration to set up the endpoint. See [POST
/api/2.0/serving-endpoints](https://docs.databricks.com/api/workspace/servingendpoints/create) for API details.

* Please keep in mind this openai config is related to an example provided please change the `openai_config` to reflect your environment

In [0]:
create_endpoint_request_data = {
    "name": ENDPOINT_NAME,
    "config": {
        "served_entities": [
            {
                "name": "gpt-4o",
                "external_model": {
                    "name": "gpt-4o",
                    "provider": "openai",
                    "task": "llm/v1/chat",
                    "openai_config": {
                        "openai_api_type": "azure",
                        "openai_api_key": f"{{{{secrets/{SECRETS_SCOPE}/{SECRETS_KEY}}}}}",
                        "openai_api_base": "https://doan-azure-openai.openai.azure.com",
                        "openai_deployment_name": "gpt-4o",
                        "openai_api_version": "2025-01-01-preview",
                    },
                },
            }
        ],
    },
}

In [0]:
import time

tmp_token = w.tokens.create(
    comment=f"sdk-{time.time_ns()}", lifetime_seconds=120
).token_value

configure_endpoint(
    ENDPOINT_NAME, tmp_token, create_endpoint_request_data, DATABRICKS_HOST
)

One of the immediate benefits of using OpenAI models (or models from other providers) using Databricks is that you can immediately query the model using the any of the following methods:
 - Databricks Python SDK
 - OpenAI Python client
 - REST API calls
 -  MLflow Deployments SDK
 - Databricks SQL `ai_query` function 

See the **Query foundation models and external models** article ([AWS](https://docs.databricks.com/en/machine-learning/model-serving/score-foundation-models.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/model-serving/score-foundation-models) |  [GCP](https://docs.databricks.com/gcp/en/machine-learning/model-serving/score-foundation-models)).

For example, you can use `ai_query` to query the model with Databricks SQL.

In [0]:
%sql
SELECT
  ai_query("gpt-4o", "What is a mixture of experts model?")

## Add an AI Gateway configuration

After you set up a model serving endpoint, you can query the OpenAI model using any of the various querying methods accessible in Databricks.

You can further enrich the model serving endpoint by enabling the Databricks Mosaic AI Gateway, which offers a variety of features for monitoring and managing your endpoint. These features include inference tables, guardrails, and rate limits, among other things.

To start, the following is a simple configuration that enables inference tables for monitoring endpoint usage. Understanding how the endpoint is being used and how often, helps to determine what usage limits and guardrails are beneficial for your use case.

In [0]:
gateway_request_data = {
    "usage_tracking_config": {"enabled": True},
    "inference_table_config": {
        "enabled": True,
        "catalog_name": CATALOG_NAME,
        "schema_name": SCHEMA_NAME,
    },
}

In [0]:
tmp_token = w.tokens.create(
    comment=f"sdk-{time.time_ns()}", lifetime_seconds=120
).token_value

configure_endpoint(
    ENDPOINT_NAME, tmp_token, gateway_request_data, DATABRICKS_HOST, "ai-gateway"
)